# Estatísticas do Controle de Processos CEP

O controle estatístico do processo ou qualidade é um sistema de verificação por amostragem, este opera no decorrer de todo o processo, seu principal objetivo é verificar a presença de caraterísticas especiais, ou seja, características que não são naturais (anomalias) ao processo e que quase certamente prejudicam a qualidade do produto ou fenômeno estudado.

Neste notebook serão implementadas todas as funções para cálculo das estatísticas de controle da qualidade, em geral, funções prontas omitem os as saídas dos resultados e provém apenas o resultado gráfico, que neste caso não é de nosso interesse.

Detalhes e pressupostos das funções podem ser encontrados em Montgomery, (2020). Os dados utilizados nestes exemplos são educativos.  


In [1]:
#libs
import pandas as pd
import numpy as np
import statsmodels.graphics.tsaplots as smt
import statsmodels.tsa.stattools as sta

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#dados
df = pd.read_csv("/home/wesley/discovery/local_files/testes/data/exemplo_nove.csv")

## Shewhart Control Chart

### Control Limits for the R Chart

LCS = UCL = $D_4  \overline{R}$

Center Line = $\overline{R}$

LCI = LCL = $D_3  \overline{R}$

### Control Limits for the X Chart

LCS = UCL = $\overline{x} + A_2  \overline{R}$

Center Line = $\overline{x}$

LCI = LCL = $\overline{x} - A_2  \overline{R}$

In [9]:
def shewhart_chart(df,m):
    factors = pd.read_csv("factors_control_chart.csv")
    
    #Control Limits for the R Chart
    r_mean = df['Ampl'].mean()
    ucl_r = factors['D4'][m-2]*r_mean
    lcl_r = factors['D3'][m-2]*r_mean
    
    #Control Limits for the X Chart
    x_mean = df['Mean'].mean()
    ucl_x = x_mean + factors['A2'][m-2]*r_mean
    lcl_x = x_mean - factors['A2'][m-2]*r_mean
    
    return(ucl_r, ucl_x)

### Control chart for individual units - Shewhart

#### Moving range control chart

$MR_i = |x_i - x_{i-1}|$


LCS = UCL = $\overline{x} + 3 \frac{\overline{MR}}{d_2}$

Center Line = $\overline{x}$

LCI = LCL = $\overline{x} - 3 \frac{\overline{MR}}{d_2}$

In [4]:
def shewhart_moving_range(xi):
    
    mr = []
    
    count = 0
    for i in xi:
        try:
            mr.append(np.abs(i-xi[count+1]))
        except KeyError:
            continue
    
    ucl = np.mean(xi) + 3*np.mean(mr)/1.128
    
    return(ucl)

## The Cumulative Sum Control Chart

### The Tabular Cusum

$C^+_i = max[0, x_i-(\mu_0+k) + C^+_{i-1}]$

$C^-_i = max[0, (\mu_0-k)-x_i + C^-_{i-1}]$

**OBS:** $C^+_{i-1}$ e $C^-_{i-1}$ são iniciados com $0$.

In [4]:
def cusum_tabular(xi, k=1/2):
    xi_mean_k_pos = np.mean(xi) + k
    xi_mean_k_neg = np.mean(xi) - k
    
    ci_pos = []
    ci_neg = []
    xi_dif_mk = []
    mk_dif_xi = []
    n_positivo = []
    n_negativo = []
    n_pos = 0
    n_neg = 0
    
    for i in xi:
        #Ci positivo
        if len(ci_pos) == 0:
            xi_dif_mk.append(i - xi_mean_k_pos)
            ci_pos.append(max(0, i - xi_mean_k_pos + 0))
            if ci_pos[-1] == 0:
                n_pos = 0
            elif ci_pos[-1] > 0:
                n_pos += 1
        else:
            xi_dif_mk.append(i - xi_mean_k_pos)
            ci_pos.append(max(0, i - xi_mean_k_pos + ci_pos[-1]))
            if ci_pos[-1] == 0:
                n_pos = 0
            elif ci_pos[-1] > 0:
                n_pos += 1
        
        #Ci negativo
        if len(ci_neg) == 0:
            mk_dif_xi.append(xi_mean_k_neg - i)
            ci_neg.append(max(0, xi_mean_k_neg - i + 0))
            if ci_neg[-1] == 0:
                n_neg = 0
            elif ci_neg[-1] > 0:
                n_neg += 1
        else:
            mk_dif_xi.append(xi_mean_k_neg - i)
            ci_neg.append(max(0, xi_mean_k_neg - i + ci_neg[-1]))
            if ci_neg[-1] == 0:
                n_neg = 0
            elif ci_neg[-1] > 0:
                n_neg += 1
                
        n_positivo.append(n_pos)
        n_negativo.append(n_neg)
    
    df_cusum = pd.DataFrame({'xi':xi, 'xi-(mean+k)':xi_dif_mk, 'ci_pos':ci_pos, 'n_pos':n_positivo,
                            '(mean+k)-xi':mk_dif_xi, 'ci_neg':ci_neg, 'n_neg':n_negativo})
    
    df_cusum = df_cusum[['xi', 'xi-(mean+k)', 'ci_pos', 'n_pos', '(mean+k)-xi', 'ci_neg', 'n_neg']]
    
    H = 5 *  np.std(xi)
    
    
    return df_cusum, H
    

In [5]:
cusum_tabular(df.x)

,xi,xi-(mean+k),ci_pos,n_pos,(mean+k)-xi,ci_neg,n_neg
0,9.45,-1.05,0.00,0,0.05,0.05,1
1,7.99,-2.51,0.00,0,1.51,1.56,2
2,9.29,-1.21,0.00,0,0.21,1.77,3
3,11.66,1.16,1.16,1,-2.16,0.00,0
4,12.16,1.66,2.82,2,-2.66,0.00,0
5,10.18,-0.32,2.50,3,-0.68,0.00,0
6,8.04,-2.46,0.04,4,1.46,1.46,1
7,11.46,0.96,1.00,5,-1.96,0.00,0
8,9.20,-1.30,0.00,0,0.30,0.30,1
9,10.34,-0.16,0.00,0,-0.84,0.00,0


### The Standardized Cusum

$y_i = \frac{x_i - \mu}{\sigma}$

$C^+_i = max[0, y_i + k + C^+_{i-1}]$

$C^-_i = max[0, -k-y_i + C^-_{i-1}]$


In [6]:
def cusum_standardized(xi, k=1/2):
    yi = (xi - np.mean(xi))/np.std(xi)
    
    ci_pos = []
    ci_neg = []
    yi_dif_mk = []
    mk_dif_yi = []
    n_positivo = []
    n_negativo = []
    n_pos = 0
    n_neg = 0
    
    for i in yi:
        #Ci positivo
        if len(ci_pos) == 0:
            yi_dif_mk.append(i - k)
            ci_pos.append(max(0, i - k + 0))
            if ci_pos[-1] == 0:
                n_pos = 0
            elif ci_pos[-1] > 0:
                n_pos += 1
        else:
            yi_dif_mk.append(i - k)
            ci_pos.append(max(0, i - k + ci_pos[-1]))
            if ci_pos[-1] == 0:
                n_pos = 0
            elif ci_pos[-1] > 0:
                n_pos += 1
        
        #Ci negativo
        if len(ci_neg) == 0:
            mk_dif_yi.append(-k - i)
            ci_neg.append(max(0, -k - i + 0))
            if ci_neg[-1] == 0:
                n_neg = 0
            elif ci_neg[-1] > 0:
                n_neg += 1
        else:
            mk_dif_yi.append(-k - i)
            ci_neg.append(max(0, -k - i + ci_neg[-1]))
            if ci_neg[-1] == 0:
                n_neg = 0
            elif ci_neg[-1] > 0:
                n_neg += 1
                
        n_positivo.append(n_pos)
        n_negativo.append(n_neg)
    
    df_cusum = pd.DataFrame({'yi':yi, 'yi-(mean+k)':yi_dif_mk, 'ci_pos':ci_pos, 'n_pos':n_positivo,
                            '(mean+k)-yi':mk_dif_yi, 'ci_neg':ci_neg, 'n_neg':n_negativo})
    
    df_cusum = df_cusum[['yi', 'yi-(mean+k)', 'ci_pos', 'n_pos', '(mean+k)-yi', 'ci_neg', 'n_neg']]
    H = 5*np.std(yi)
    
    return df_cusum, H 
    

In [8]:
a, b = cusum_standardized(df.x, k=1/2)
a

,yi,yi-(mean+k),ci_pos,n_pos,(mean+k)-yi,ci_neg,n_neg
0,-0.762687,-1.262687,0.000000,0,0.262687,0.262687,1
1,-2.049997,-2.549997,0.000000,0,1.549997,1.812684,2
2,-0.903762,-1.403762,0.000000,0,0.403762,2.216446,3
3,1.185912,0.685912,0.685912,1,-1.685912,0.530534,4
4,1.626772,1.126772,1.812684,2,-2.126772,0.000000,0
5,-0.119032,-0.619032,1.193652,3,-0.380968,0.000000,0
6,-2.005911,-2.505911,0.000000,0,1.505911,1.505911,1
7,1.009568,0.509568,0.509568,1,-1.509568,0.000000,0
8,-0.983117,-1.483117,0.000000,0,0.483117,0.483117,1
9,0.022043,-0.477957,0.000000,0,-0.522043,0.000000,0


## The Exponentially Weighted Moving Average Control Chart

### The exponentially weighted moving average is defined as:

$z_i = \lambda x_i + (1 - \lambda) z_{i-1} $

where $0 < \lambda ≤ 1$ is a constant and the starting value (required with the first sample at $i = 1$) is
the process target, so that

$z_0 = \mu_0$

Sometimes the average of preliminary data is used as the starting value of the EWMA, so that $z_0 = \bar{x} $.

### The EWMA control chart

UCL = $ \mu_0 + L\sigma \sqrt{\frac{\lambda}{(2-\lambda)}[1-(1-\lambda)^{2i}]}$

Center Line = $\mu_0$

LCL = $ \mu_0 - L\sigma \sqrt{\frac{\lambda}{(2-\lambda)}[1-(1-\lambda)^{2i}]}$

In [76]:
def ewma_chart(xi, lambd = 0.1, l = 2.7):
    
    xi_mean = np.mean(xi)
    xi_std = np.std(xi)
    yi = []
    count = 1
    ucl = []
    lcl = [] 
    
    for i in xi:
        if len(yi) == 0:
            yi.append(lambd*i + (1-lambd)*xi_mean)
            ucl.append(xi_mean + l * xi_std * np.sqrt((lambd/(2-lambd))*(1-(1-lambd)**(2*count))))
            lcl.append(xi_mean - l * xi_std * np.sqrt((lambd/(2-lambd))*(1-(1-lambd)**(2*count))))
            count += 1
        else:
            yi.append(lambd*i + (1-lambd)*yi[-1])
            ucl.append(xi_mean + l * xi_std * np.sqrt((lambd/(2-lambd))*(1-(1-lambd)**(2*count))))
            lcl.append(xi_mean - l * xi_std * np.sqrt((lambd/(2-lambd))*(1-(1-lambd)**(2*count))))
            count += 1
    df_ewma = pd.DataFrame({'xi':xi, 'yi':yi, 'ucl':ucl, 'lcl':lcl})
    H = 5 * xi_std
    
    return df_ewma, H

In [22]:
ewma_chart(df.x)

,lcl,ucl,xi,yi
0,10.008780,10.621220,9.45,10.228500
1,9.903023,10.726977,7.99,10.004650
2,9.834118,10.795882,9.29,9.933185
3,9.784829,10.845171,11.66,10.105867
4,9.748037,10.881963,12.16,10.311280
5,9.719901,10.910099,10.18,10.298152
6,9.698051,10.931949,8.04,10.072337
7,9.680904,10.949096,11.46,10.211103
8,9.667348,10.962652,9.20,10.109993
9,9.656571,10.973429,10.34,10.132993


## The Batch Means Control Chart

Runger and Willemain (1996) proposed a control chart based on unweighted batch means for monitoring autocorrelated process data. The batch means approach has been used extensively in the analysis of the output from computer
simulation models, another area where highly autocorrelated data often occur. The
unweighted batch means (UBM) control chart breaks successive groups of sequential
observations into batches, with equal weights assigned to every point in the batch. Let the jth
unweighted batch mean be

$\bar{x}_j = \frac{1}{b} \sum^b_{i=j} x_{(j-1)b+i}    \;\;\;j = 1,2 \dots$



In [73]:
def batch_means(xi, b):
    # xi e um vetor contendo valores da variavel de interesse
    # b e o tamanho do batch, o melhor batch e aquele que reduz a autocorrelacao no lag 1
    
    paf = sta.pacf(xi, nlags=int(len(xi)-len(xi)*0.4), alpha=0.5)[0][1] #partial autocorrelation function
    
    if paf < 0.2:
        
        return xi
    
    else:
        count = 1
        while (paf >= 0.2) and (count <= 4):    
            yi = []
            n = len(xi)
            wind_inf = 0
            wind_sup = wind_inf + b 

            while wind_sup <= n:
                yi.append(np.mean(xi[wind_inf:wind_sup]))
                wind_inf += b
                wind_sup += b
            
            b = b * 2
            paf = sta.pacf(yi, nlags=int(len(yi)-len(yi)*0.4), alpha=0.5)[0][1]
            count += 1
    
    return yi

In [74]:
yi = batch_means(df.x, 2)

In [77]:
ewma_chart(yi)

,lcl,ucl,xi,yi
0,10.008780,10.621220,9.45,10.228500
1,9.903023,10.726977,7.99,10.004650
2,9.834118,10.795882,9.29,9.933185
3,9.784829,10.845171,11.66,10.105867
4,9.748037,10.881963,12.16,10.311280
5,9.719901,10.910099,10.18,10.298152
6,9.698051,10.931949,8.04,10.072337
7,9.680904,10.949096,11.46,10.211103
8,9.667348,10.962652,9.20,10.109993
9,9.656571,10.973429,10.34,10.132993


## Referência

MONTGOMERY, Douglas C. Introduction to statistical quality control. John Wiley & Sons, 2020.